# Doc2Vecの精度確認

## (1) 確認時のDoc2Vecパラメータ等

### (1-1) 使用した文書

Wikipedia文書、my-ope文書の双方を使用し、ボキャブラリ／単語ベクトルの生成および学習を行い、モデルを生成しました。

レポート <a href="25-Create-doc2vec-model-wiki-myope.ipynb"><b>25-Create-doc2vec-model-wiki-myope.ipynb</b></a> の手順にて生成しています。

- コーパスが広大すぎてmy-ope文書と単語数が合わない為、Wikipedia文書は、句読点で１文書と扱う様にしています。


- 説明文が非常に長いWikipedia文書は、全てを取り込むとタイトルと説明文との関連性が薄くなってしまう為、先頭から１０件の説明文だけを取り込む様にしています。

### (1-2) モデルパラメータ

下記の様にパラメータを指定しています。

In [ ]:
model = Doc2Vec(documents=tagged_document_list, dm=0, size=200, 
                    window=20, min_count=1, iter=size*10, workers=100)

## (2) TF-IDF vs Doc2Vecでaccuracy比較

為念で３回試行しています。大きな変化は見られませんでした。

| テストデータ | サンプル数 | accuracy（TF-IDF使用時） | <> | accuracy（Doc2Vec使用時） |
| :--- | :---: | :---: | :---: | :---: | :---: | --- |
| question_answers/toyotsu_human.csv | 317 | 0.975 | < | 0.978 |


## (3) 文書のパターンによる精度の変化

> 同じ意味だけど、異なる言い回しや単語を使用しているような文章で類似検索してみる

>「〜ない」のような意味が逆転するfeatureを持つ文章をコサイン類似検索

レポート <a href="26-Doc2Vec-Wiki-with-scikit_learn.ipynb"><b>26-Doc2Vec-Wiki-with-scikit_learn.ipynb</b></a> の手順にて検証しております。

ここでは結果のダイジェストを掲載します。

### (3-1) 同じ意味だが言い回しが違う文書

In [ ]:
# 正解＝6763
search_simiarity('海外の出張費の精算の方法は？', loaded_model_dbow)

```
   answer_id  question_answer_id  similarity
0       6763             13335.0    0.807660
```

In [ ]:
# 正解＝6763（違った言い回し）
search_simiarity('海外出張時の費用を精算したいのですが', loaded_model_dbow)

残念ながら、正解を得ることはできませんでした（similarityは上から９番目）。

```
   answer_id  question_answer_id  similarity
0       6856             13430.0    0.796510
1       6802             13377.0    0.795700
2       6795             13370.0    0.780811
3       6790             13364.0    0.769724
4       6824             13398.0    0.765216
5       6902             13476.0    0.760034
6       6873             13447.0    0.758189
7       6762             13334.0    0.754221
8       6763             13335.0    0.751823
```

学習セットのmy-ope文書に、この文書のバリエーションが１件しかないのと、上記のような言い回しのWikipedia文書が少ないため、言い回しの違いにうまく対応できていないものと考えております。

### (3-2) 意味が逆転するfeatureを持つ文書

あらかじめ、意味が逆転するfeatureを持つ文書を、学習セットに追加しておけば、対応できそうなことは確認できました。

In [ ]:
# 正解＝6975
search_simiarity('退職金の振込口座を変えたい', loaded_model_dbow) 

```
   answer_id  question_answer_id  similarity
0       6975             13555.0    0.990793
1       7050             13700.0    0.965827
2       6997             13577.0    0.795659
```

In [ ]:
# 正解＝7050（6975の反対として設定）
search_simiarity('退職金の振込口座を変えたくない', loaded_model_dbow) 

```
   answer_id  question_answer_id  similarity
0       7050             13700.0    0.990555
1       6975             13555.0    0.957918
2       6997             13577.0    0.790626
```